In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [ ]:
# #Instantiate Model
# tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("csatapathy/interview-ratings-bert")
model = AutoModelForSequenceClassification.from_pretrained("csatapathy/interview-ratings-bert")

tokenizer_config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
#Encode and Calculate Sentiment
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')


In [ ]:
result = model(tokens)

In [ ]:
result.logits



tensor([[-0.9703, -1.6689, -2.0793, -0.8761,  0.0156,  1.5680,  2.3805,  1.6150,
         -0.5132, -1.5832]], grad_fn=<AddmmBackward0>)

In [ ]:
int(torch.argmax(result.logits))+1

7

In [ ]:
# Loading Answers into DataFrame and Score

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
reviews_df = pd.read_parquet('/content/0000.parquet')

In [ ]:
reviews_df

,instruction,input,output,instruction_length,output_length,title,start
0,What are the two stages of a startup's journey...,,Post product market fit and pre product market...,17,1119,The Secrets to YC's Essential Startup Advice w...,Post product market fit and pre product market...
1,How can startups improve their product and lea...,,"Faster that you iterate, the faster you learn ...",18,452,The Secrets to YC's Essential Startup Advice w...,"Faster that you iterate, the faster you learn ..."
2,How do YC startups compete against larger incu...,,Interesting things that we've learned is that ...,18,549,The Secrets to YC's Essential Startup Advice w...,Interesting things that we've learned is that ...
3,What is one aspect of startup advice that is o...,,Smart you are. And there's almost no talk abou...,19,423,The Secrets to YC's Essential Startup Advice w...,Smart you are. And there's almost no talk abou...
4,What factors do you think contribute to the su...,,Look at your peer group in university or in hi...,20,906,The Secrets to YC's Essential Startup Advice w...,Look at your peer group in university or in hi...
...,...,...,...,...,...,...,...
549,What is the economic argument for doing a star...,,"Number 4, the economic argument. Doing a start...",17,35,Building Minimal Viable Product with Michael S...,"Number 4, the economic argument. Doing a start..."
550,Can you please provide an overview of the agen...,,"So agenda, I'll go over logistics real quick, ...",16,116,Building Minimal Viable Product with Michael S...,"So agenda, I'll go over logistics real quick, ..."
551,What is your opinion on the effectiveness of s...,,Your opinion like what makes an effective soci...,16,758,Building Minimal Viable Product with Michael S...,Your opinion like what makes an effective soci...
552,What is the typical range of investment for se...,,Seed funds are typically funds that are betwee...,25,610,Building Minimal Viable Product with Michael S...,Seed funds are typically funds that are betwee...


In [ ]:
len(reviews_df)

554

In [ ]:
reviews = reviews_df['output'].tolist()

In [ ]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [ ]:
df['review'].iloc[0]

"Post product market fit and pre product market fit. So the game when you're pre product market fit is how little can you spend? How much of your company can you own? To reach the product market fit. The second place where we see this screwing up time and time again is when companies make a model from how their post seed round is gonna be spent. They only model out costs, they never model out revenue. So they come back to us and they say, well, we have to raise more money because we wanna hire these 3 engineers, and we have to get this designer, we need this, and it's gonna cost all this money. And we always come back and we say, okay, well, during this entire time that you're spending this money, you know, you're spending your c run. Are you going to make any money? Are you going to generate any revenue? Have you modeled how much revenue you're gonna generate? Have you thought about whether you should continue to hire people if you're not generating any revenue? You're not turning in 

In [ ]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
sample_size = 50

In [ ]:
sampled_df = df.sample(n=sample_size, random_state=42)

In [ ]:
sentiment_score(sampled_df['review'].iloc[1])

4

In [ ]:
sampled_df['sentiment'] = sampled_df['review'].apply(lambda x: sentiment_score(x[:512]))

In [ ]:
sampled_df

,review,sentiment
83,"Says, good problems, they're popular. So a lot...",7
81,YC only funds companies who have tons and tons...,4
265,Your company. But there's 3 critical phases to...,4
463,Once you're around smart people and you're wor...,4
165,Side. Okay. Let's go to a different thing here...,5
549,"Number 4, the economic argument. Doing a start...",4
131,You need at least some idea generators. There ...,8
86,"Was my startup, and it's an online form and su...",8
137,"Second. While there's no single archetype, gre...",7
412,"To be really, really aligned on that. When, yo...",4


In [ ]:
df['review'].iloc[3]

"Smart you are. And there's almost no talk about discipline. And I see founders almost embracing this by trying to communicate how well read they are or what schools they went to and, like, all these other things to try to signal intelligence because the way that Silicon Valley kind of shows off its most successful people is by calling them the smartest, right? Elon Musk is a smart assert. Mark Zuckerberg is smart. I actually think this is another extremely gross mischaracterization about what's going on. Almost everyone who's watching this talk right now and who will watch in the future will have gotten almost every piece of advice I'd given. They should take a second right now to ask themselves which of these pieces of advice they're not following and why? 1 of the things that I did in the beginning of COVID was start to ask myself Why don't I have better discipline in my life? I was working extremely hard and I wasn't really enjoying my day to day life and I was asking myself why no